# Meta Llama Synthetic Data Kit - Yerel Model Kullanım Rehberi

Bu notebook, Meta Llama'nın Synthetic Data Kit'ini yerel Llama modelinizle nasıl kullanacağınızı adım adım gösterir.

## İçindekiler
1. Synthetic Data Kit Nedir?
2. Kurulum
3. Yerel Model API Ayarları (vLLM)
4. Temel Kullanım - 4 Adım
   - Ingest (Veri Yükleme)
   - Create (Sentetik Veri Oluşturma)
   - Curate (Veri Filtreleme)
   - Save-As (Format Dönüştürme)
5. Pratik Demo

---

## 1. Synthetic Data Kit Nedir?

Meta Llama'nın Synthetic Data Kit'i, LLM'leri fine-tune etmek için yüksek kaliteli sentetik veri setleri oluşturmaya yarayan açık kaynaklı bir araçtır.

**Ana Özellikler:**
- PDF, HTML, DOCX, TXT gibi çeşitli dosya formatlarını işler
- QA çiftleri, Chain-of-Thought, özet gibi formatlar üretir
- Kalite kontrolü için LLM-as-a-judge kullanır
- Yerel model desteği (vLLM) ve API endpoint desteği
- Toplu işlem (batch processing) yetenekleri

## 2. Kurulum

### Gereksinimler
- Python 3.10+
- Yerel Llama modeli (örn: Llama 3.1, Llama 3.2)
- vLLM (yerel inference için)

In [ ]:
# Synthetic Data Kit'i kuralım
!pip install synthetic-data-kit

In [ ]:
# vLLM kurulumu (yerel model çalıştırmak için)
!pip install vllm

## 3. Yerel Model API Ayarları (vLLM)

Synthetic Data Kit, yerel modelinizi kullanmak için bir API endpoint'e ihtiyaç duyar. vLLM ile yerel modelinizi OpenAI-uyumlu bir API olarak çalıştırabilirsiniz.

### 3.1. vLLM Sunucusunu Başlatma

**Terminal'de çalıştırın (ayrı bir terminal penceresi):**

```bash
# Llama modelinizin yolunu belirtin
python -m vllm.entrypoints.openai.api_server \
    --model /path/to/your/llama-model \
    --host 0.0.0.0 \
    --port 8000 \
    --dtype auto \
    --api-key your-api-key-here

# Örnek:
python -m vllm.entrypoints.openai.api_server \
    --model meta-llama/Llama-3.2-3B-Instruct \
    --host 0.0.0.0 \
    --port 8000 \
    --dtype auto \
    --api-key sk-123456
```

### 3.2. API Bağlantısını Test Etme

In [ ]:
import requests
import json

# vLLM API'nizin çalıştığını test edin
api_url = "http://localhost:8000/v1/chat/completions"
api_key = "sk-123456"  # vLLM başlatırken kullandığınız API key

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

data = {
    "model": "meta-llama/Llama-3.2-3B-Instruct",  # Model adınız
    "messages": [
        {"role": "user", "content": "Merhaba! Nasılsın?"}
    ],
    "max_tokens": 100
}

try:
    response = requests.post(api_url, headers=headers, json=data)
    response.raise_for_status()
    result = response.json()
    print("✅ API başarıyla çalışıyor!")
    print(f"Model yanıtı: {result['choices'][0]['message']['content']}")
except Exception as e:
    print(f"❌ API hatası: {e}")
    print("Lütfen vLLM sunucusunun çalıştığından emin olun.")

## 4. Synthetic Data Kit - 4 Adımlık İş Akışı

Synthetic Data Kit 4 ana komuttan oluşur:

1. **`ingest`**: Dokümanları işler ve parse eder
2. **`create`**: Sentetik eğitim örnekleri oluşturur (QA, CoT, özet)
3. **`curate`**: Kalite kontrolü yapar ve filtreler
4. **`save-as`**: Fine-tuning formatlarına dönüştürür

### 4.1. Konfigürasyon Dosyası Oluşturma

Yerel API'nizi kullanmak için bir config dosyası oluşturalım:

In [ ]:
import yaml
import os

# Demo için klasör yapısı oluşturalım
os.makedirs('demo_data/input', exist_ok=True)
os.makedirs('demo_data/output', exist_ok=True)

# Konfigürasyon dosyası
config = {
    'llm': {
        'provider': 'openai',  # vLLM OpenAI-uyumlu API kullanır
        'api_base': 'http://localhost:8000/v1',
        'api_key': 'sk-123456',  # vLLM API key'iniz
        'model': 'meta-llama/Llama-3.2-3B-Instruct',  # Model adınız
        'temperature': 0.7,
        'max_tokens': 2048
    },
    'ingest': {
        'chunk_size': 2000,
        'chunk_overlap': 200
    },
    'create': {
        'format': 'qa',  # qa, cot, summary
        'num_examples': 5
    },
    'curate': {
        'quality_threshold': 0.7
    }
}

# Config dosyasını kaydet
with open('demo_data/config.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("✅ Konfigürasyon dosyası oluşturuldu: demo_data/config.yaml")
print("\nİçerik:")
print(yaml.dump(config, default_flow_style=False))

### 4.2. Örnek Veri Hazırlama

Demo için örnek bir metin dosyası oluşturalım:

In [ ]:
# Örnek metin verisi
sample_text = """
Yapay Zeka ve Makine Öğrenmesi

Yapay zeka (AI), makinelerin insan benzeri zeka gerektiren görevleri yerine getirmesini sağlayan 
bir bilgisayar bilimi dalıdır. Makine öğrenmesi ise yapay zekanın bir alt dalıdır ve makinelerin 
deneyimlerden öğrenmesini ve zamanla performansını artırmasını sağlar.

Derin öğrenme, makine öğrenmesinin özel bir türüdür ve yapay sinir ağlarını kullanır. Bu ağlar, 
insan beyninin çalışma şeklinden ilham alır ve birden fazla katmandan oluşur. Her katman, veriden 
farklı seviyede özellikler öğrenir.

Large Language Models (LLM'ler), milyarlarca parametre içeren ve büyük metin veri setleri üzerinde 
eğitilen derin öğrenme modelleridir. Bu modeller, doğal dil işleme görevlerinde çığır açmıştır. 
GPT, BERT ve Llama gibi modeller, metin oluşturma, çeviri, soru-cevap ve birçok diğer görevde 
kullanılır.

Fine-tuning, önceden eğitilmiş bir modelin belirli bir görev için yeniden eğitilmesi sürecidir. 
Bu işlem, modelin genel bilgisini korurken, özel bir domaine veya göreve adapte olmasını sağlar.
"""

with open('demo_data/input/ai_tutorial.txt', 'w', encoding='utf-8') as f:
    f.write(sample_text)

print("✅ Örnek veri dosyası oluşturuldu: demo_data/input/ai_tutorial.txt")

## 5. Pratik Demo - Adım Adım Kullanım

Şimdi Synthetic Data Kit'in tüm adımlarını kullanalım.

### Adım 1: INGEST - Dokümanları İşleme

İlk adım, ham dokümanları parse edip chunk'lara ayırmaktır.

In [ ]:
# INGEST komutu
!synthetic-data-kit ingest \
    --input-dir demo_data/input \
    --output-file demo_data/ingested.jsonl \
    --chunk-size 1000 \
    --chunk-overlap 100

In [ ]:
# İşlenmiş veriyi inceleyelim
import json

print("📄 İşlenmiş Veri:")
print("=" * 50)

with open('demo_data/ingested.jsonl', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f, 1):
        data = json.loads(line)
        print(f"\nChunk {i}:")
        print(f"Kaynak: {data.get('source', 'N/A')}")
        print(f"İçerik (ilk 200 karakter): {data.get('text', '')[:200]}...")
        print("-" * 50)

### Adım 2: CREATE - Sentetik Eğitim Verisi Oluşturma

İşlenmiş veriden QA çiftleri, Chain-of-Thought veya özet örnekleri oluşturalım.

**ÖNEMLİ:** Bu adım yerel modelinizi kullanacak. vLLM sunucunuzun çalıştığından emin olun!

In [ ]:
# CREATE komutu - QA çiftleri oluşturma
!synthetic-data-kit create \
    --input-file demo_data/ingested.jsonl \
    --output-file demo_data/created_qa.jsonl \
    --format qa \
    --api-base http://localhost:8000/v1 \
    --api-key sk-123456 \
    --model meta-llama/Llama-3.2-3B-Instruct \
    --temperature 0.7 \
    --num-examples 3

In [ ]:
# Oluşturulan QA çiftlerini inceleyelim
print("💬 Oluşturulan QA Çiftleri:")
print("=" * 50)

with open('demo_data/created_qa.jsonl', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f, 1):
        data = json.loads(line)
        print(f"\n{'='*50}")
        print(f"Örnek {i}:")
        print(f"\nSoru: {data.get('question', 'N/A')}")
        print(f"\nCevap: {data.get('answer', 'N/A')}")
        print(f"\nKaynak: {data.get('source', 'N/A')}")

### Alternatif: Chain-of-Thought (CoT) Formatında Oluşturma

In [ ]:
# CREATE komutu - CoT formatı
!synthetic-data-kit create \
    --input-file demo_data/ingested.jsonl \
    --output-file demo_data/created_cot.jsonl \
    --format cot \
    --api-base http://localhost:8000/v1 \
    --api-key sk-123456 \
    --model meta-llama/Llama-3.2-3B-Instruct \
    --temperature 0.7 \
    --num-examples 2

### Adım 3: CURATE - Kalite Kontrolü

LLM-as-a-judge kullanarak düşük kaliteli örnekleri filtreleyelim.

In [ ]:
# CURATE komutu
!synthetic-data-kit curate \
    --input-file demo_data/created_qa.jsonl \
    --output-file demo_data/curated.jsonl \
    --api-base http://localhost:8000/v1 \
    --api-key sk-123456 \
    --model meta-llama/Llama-3.2-3B-Instruct \
    --quality-threshold 0.7

In [ ]:
# Filtrelenmiş veriyi inceleyelim
print("🔍 Kalite Kontrolü Sonuçları:")
print("=" * 50)

with open('demo_data/curated.jsonl', 'r', encoding='utf-8') as f:
    curated_data = [json.loads(line) for line in f]

print(f"\nFiltrelenmiş örnek sayısı: {len(curated_data)}")

for i, data in enumerate(curated_data, 1):
    print(f"\n{'='*50}")
    print(f"Örnek {i}:")
    print(f"Kalite Skoru: {data.get('quality_score', 'N/A')}")
    print(f"Soru: {data.get('question', 'N/A')}")
    print(f"Cevap: {data.get('answer', 'N/A')[:200]}...")

### Adım 4: SAVE-AS - Fine-tuning Formatına Dönüştürme

Son olarak, verimizi fine-tuning için uygun formatlara dönüştürelim.

In [ ]:
# SAVE-AS komutu - Alpaca formatı
!synthetic-data-kit save-as \
    --input-file demo_data/curated.jsonl \
    --output-file demo_data/final_alpaca.json \
    --format alpaca

In [ ]:
# Alpaca formatını inceleyelim
with open('demo_data/final_alpaca.json', 'r', encoding='utf-8') as f:
    alpaca_data = json.load(f)

print("📦 Alpaca Formatı:")
print("=" * 50)
print(f"Toplam örnek: {len(alpaca_data)}\n")

for i, item in enumerate(alpaca_data[:2], 1):  # İlk 2 örneği göster
    print(f"\nÖrnek {i}:")
    print(json.dumps(item, indent=2, ensure_ascii=False))
    print("-" * 50)

In [ ]:
# ChatML formatına da dönüştürebiliriz
!synthetic-data-kit save-as \
    --input-file demo_data/curated.jsonl \
    --output-file demo_data/final_chatml.jsonl \
    --format chatml

print("✅ ChatML formatı oluşturuldu: demo_data/final_chatml.jsonl")

## 6. Python ile Programatik Kullanım

CLI yerine Python kodu ile de kullanabilirsiniz:

In [ ]:
# Python API kullanımı (varsa)
# Not: Bu kısım SDK'nın Python API'sine göre güncellenmelidir

from openai import OpenAI

# Yerel modelinize bağlanma
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="sk-123456"
)

# Örnek kullanım
response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-3B-Instruct",
    messages=[
        {"role": "system", "content": "Sen yardımcı bir asistansın."},
        {"role": "user", "content": "Yapay zeka nedir?"}
    ],
    max_tokens=200
)

print("Model Yanıtı:")
print(response.choices[0].message.content)

## 7. Toplu İşleme (Batch Processing)

Birden fazla dosyayı aynı anda işleyebilirsiniz:

In [ ]:
# Birden fazla dosya ekleyelim
texts = [
    ("python_basics.txt", "Python bir programlama dilidir. Basit sözdizimine sahiptir."),
    ("database.txt", "Veritabanları veri depolamak için kullanılır. SQL sorgu dilidir.")
]

for filename, content in texts:
    with open(f'demo_data/input/{filename}', 'w', encoding='utf-8') as f:
        f.write(content)

print("✅ Birden fazla dosya eklendi")

In [ ]:
# Tüm klasörü işleme
!synthetic-data-kit ingest \
    --input-dir demo_data/input \
    --output-file demo_data/batch_ingested.jsonl \
    --chunk-size 1000

## 8. İpuçları ve En İyi Uygulamalar

### 8.1. Model Seçimi
- **Küçük modeller (3B-8B)**: Hızlı prototipleme için
- **Orta modeller (13B-30B)**: Daha iyi kalite
- **Büyük modeller (70B+)**: En yüksek kalite ama daha yavaş

### 8.2. Parametreler
- **Temperature**: 0.7-0.9 arası çeşitlilik için iyi
- **Chunk Size**: Dökümana göre ayarlayın (1000-2000 karakter)
- **Quality Threshold**: 0.7-0.8 dengeli filtreleme

### 8.3. Performans
- GPU kullanın (vLLM ile)
- Batch işleme kullanın
- Önce küçük örneklerle test edin

### 8.4. Kalite Kontrolü
- Her adımdan sonra çıktıları inceleyin
- Curate adımını mutlaka kullanın
- Manuel olarak örnekleri gözden geçirin

## 9. Troubleshooting

### Problem: "Connection refused" hatası
**Çözüm**: vLLM sunucusunun çalıştığından emin olun:
```bash
curl http://localhost:8000/v1/models
```

### Problem: "Out of memory" hatası
**Çözüm**: 
- Daha küçük model kullanın
- Chunk size'ı azaltın
- GPU belleğini artırın

### Problem: Düşük kaliteli örnekler
**Çözüm**:
- Temperature değerini ayarlayın
- Daha büyük model kullanın
- Quality threshold'u yükseltin

## 10. Sonraki Adımlar

Artık sentetik veri ürettiniz! Şimdi yapabilecekleriniz:

1. **Fine-tuning**: Oluşturduğunuz veri ile modelinizi fine-tune edin
2. **Veri İyileştirme**: Daha fazla kaynak doküman ekleyin
3. **Format Denemeleri**: QA, CoT, Summary formatlarını deneyin
4. **Ölçeklendirme**: Daha büyük veri setleri oluşturun

### Faydalı Kaynaklar
- [Synthetic Data Kit GitHub](https://github.com/meta-llama/synthetic-data-kit)
- [vLLM Dokümantasyonu](https://docs.vllm.ai/)
- [Llama Models](https://github.com/meta-llama)

## 11. Özet

Bu notebook'ta öğrendikleriniz:

✅ Synthetic Data Kit kurulumu  
✅ vLLM ile yerel model API'si kurma  
✅ 4 adımlık veri oluşturma süreci (Ingest → Create → Curate → Save-As)  
✅ Farklı formatlar (QA, CoT, Summary)  
✅ Kalite kontrolü ve filtreleme  
✅ Fine-tuning formatlarına dönüştürme  

**Başarılar! 🎉**